# **Generation Experiment 2: Parent-Document retriever**

By default, retrievers retrieve small relevant chunks. However, passing more relevant information to the retriever about these topics to the LLM could be beneifical to the user. This maks Parent-Document retievers a potential value-add. Parent-Document retrievers first fetches the small chunks. It then looks up the parent ids for those small chunks and returns those larger (parent) chunks.

However we need to control how much context we are passing to the LLM due to the <u>context-stuffing problem</u>. Stuffing too much context into the LLM reduces the LLM's recall performance (ability to find information placed in its context window). Additionaly, the LLM will then be less likely to follow instructions.






In [ ]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark qdrant-client langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 62.1 MB/s eta 0:00:0

In [ ]:
from langchain_core.documents import Document
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

**Setup Simple Experiment Data for Experiment 4**

We first set up simple experiment data where we have 3 documents talking about different 3 different topics. This is done to have an easier <u>preliminary understanding and visualisation</u> of the behaviour of the Parent-Document retrievers as we are able to quickly look at what are the parent chunks beings retrieved

**TODO**: Move experiment data to a folder as its too long or maybe just leave it

In [ ]:
# Simple experiment example data
docs = [
    Document(
        page_content='''
        10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves breakfast, lunch, freshly baked pizzas, wine, and forest-inspired dinners. Hilltop Forest is a perfect minivacation destination, just a under an hour-drive from Helsinki. Eat, sleep and repeat.; 2. The Torby – Fiskars village, Helsinki region: The brand new Torby Hotel, opened in June 2023, is nestled in the charming Fiskars Village. The Torby offers an enchanting retreat for travellers seeking tranquillity and inspiration. Set amidst the lush countryside of the Helsinki region, this boutique hotel seamlessly blends modern design with the rich cultural heritage of the Fiskars village. The village is a treasure trove of creativity, with its vibrant art studios, quaint shops, and idyllic gardens. Guests can explore the local craftsmanship, indulge in delicious Finnish cuisine, or simply immerse themselves in the serene ambiance. Whether seeking a peaceful retreat or a cultural adventure, the Torby Hotel and Fiskars Village promise an unforgettable experience.; 3. Hotel Vihannonkulma – Loimaa, Coast and Archipelago: The Hotel Vihannonkulma building, a private home of the car dealer Olli Vihanto, was closed for almost 30 years until the apartment was rediscovered and revived into a boutique hotel opened in 2022. Spend the night in a classic room with sleek Alvar Aalto inspired lines, or in a modern 70s-style room with eccentric details and teak furniture. Enjoy the small-town atmosphere of Loimaa and stroll around in cafés, shops, and historical buildings. Loimaa, located 50 minutes northeast of Turku, serves as a gateway to the surrounding forests and nature.; 4. Kanava Resort – Oravi, Lakeland: Located on a narrow headland surrounded by Lake Saimaa, Kanava Resort is a haven for happiness seekers and outdoor adventurers. The resort is a harmonious blend of contemporary design and natural beauty. The buildings, constructed with sustainable materials, seamlessly blend into the landscape, providing an immersive experience for guests. The resort offers a range of activities across the year. In the winter, take a ride on a snowmobile or slide down the terrain on a pair of skis. During summer hop on a canoe, grab the oars and slide through the waters. Make sure not to miss the most popular Lakeland adventure: spotting ringed seals together with a guide. Kanava Resort is just a short drive away from the national parks Linnansaari and Kolovesi. While you are in the area, remember to also visit the historical landmark Olavinlinna Castle and the iconic Punkaharju Ridge.; 5. PihlasResort – Joroinen, Lakeland: Located on the shore of the biggest lake in Finland, Lake Saimaa, PihlasResort is the perfect place to embrace luxury. The brand new suites, opening in summer 2023, are equipped with a kitchen, a private sauna, and a terrace facing a private garden. Relax by the lake, have a drink in a jacuzzi, or feel the rejuvenating power of an authentic Finnish sauna – relaxation is guaranteed. During your visit, dine in the resort's own restaurant Siimes, or enjoy a unique tea menu in the nearby TeaHouse of Wehmais.; 6. Villipeura’s Niliaitta – Kivijärvi, Lakeland: At Villipeura's holiday village in Kivijärvi, located less than two hours north of Jyväskylä in central Lakeland, unique birdhouse-like boxes are elevated on pillars in the midst of a charming Finnish forest. These uncommon holiday cottages draw inspiration from traditional food storage sheds that were built on wooden pillars to protect food from animals in the past. In the holiday village area, you have access to a range of activities, including tennis, frisbee golf, and fishing. Nature trails of the national parks Salamajärvi and Pyhä Häkki are just around the corner.; 7. Glass huts in Skyfire Village – Rovaniemi, Lapland: At Skyfire Village in Rovaniemi you may need to pinch yourself to figure out whether the view is real: the panoramic windows in the glass huts allow you to gaze at northern lights in autumn and winter, or the midnight sun in the summer directly from your bed. Skyfire Village is in a quiet location with no light pollution making it even easier to see the auroras. The village's own restaurant, Sky Hut Restaurant & Bar, offers tailor-made menus by a professional chef using local ingredients.; 8. Kurula’s – Pyhätunturi, Lapland: Kurula’s in Lapland's pristine Pyhä region is the perfect vacation getaway for those who want a combination of quiet relaxation and outdoors activities. In the summer, go on a hike in Finland’s oldest national park Pyhä-Luosto or ascend the fell with a skiing lift to admire the views. In the winter, embark on a husky or reindeer safari and hunt the northern lights. Your own private lakeside sauna in the suite is a perfect way to end a day full of adventures. Kurula is close to the popular outdoor and skiing resort of Pyhä.; 9. Arctra – Rovaniemi, Lapland: At Arctra's private luxury resort near Rovaniemi in Lapland, you can enjoy a glamorous vacation atop the mighty hill of Ollerovaara, with stunning views of arctic nature. Explore the landscape by foot, on skis, or on a husky sled. A private 20 hectare wilderness area will guarantee that there most likely won’t be another soul in sight. Cosy up in your stylish suite or at the spa area. Arctra is only a 15-minute drive from Rovaniemi airport. The cottage serves as an excellent home base to explore Rovaniemi as well as the surrounding wilderness.; 10. Cahkal Hotel – Kilpisjärvi, Lapland: Cahkal Hotel is located 400 kilometres above the Arctic Circle in the northwesternmost point of Finland, in the truly unique Nordic wilderness. Cahkal is an ideal destination for experiential travellers looking for a view – all the hotel rooms have exquisite mountain views. Embark on an expedition with a local guide to explore the arctic landscape and even pop by in Norway to see the fjords and the Arctic Ocean. Unwind in the hotel's sauna after an adventure of a lifetime. Cahkal's heating system operates by sustainable geothermal heating and solar panels provide visitors with green electricity.
        '''
    ),
    Document(
        page_content='''
        Finnish food culture Finland’s cuisine is built around fresh, natural ingredients gathered straight from the waters, fields and forests: Get to know the local ingredients and delicious dishes you simply can’t miss when visiting Finland. Finns are known as leaders of research and innovation. But when it comes to food, they tend to favour tradition. Finnish cuisine differs from neighbouring Scandinavian countries because it has one foot on either side of its borders – Russia in the East and Sweden in the West. Since day one, the Finnish diet has been built around surviving the harsh conditions of our northern climate. Growth seasons are dictated by the cold, meaning many local products are only available for a limited amount of time. These days, you can buy just about anything your heart desires in Finland. But go local while you’re here, and you’ll be in for a treat! Also, if you have the chance, continue your culinary journey to some of the best restaurants in the country.; Cereals and grains – made from scratch: All hail the golden crop of Finnish soil, oats! And what’s not to love about the latest boom of oat milks, creams and yoghurts, not to mention pulled oats, which are an extremely tasty alternative to meat? Make sure to taste this decidedly Finnish innovation! If you’re more into traditional foods, stick to good ol’ porridge. This hearty delight can be enjoyed at any gas station, hotel or café for just a few euros. Warming, filling and incredibly tasty, it’s best served with a spoonful of jam or a little fresh butter. Tip: When it comes to local grains, think bakery and check out pastry chef Teemu Aura. The bakeries carrying his name and signature pink bun tracks are called Pullabiili, and they can be found throughout the Helsinki region offering classic Finnish buns, flaky croissants and sourdough breads. All these treats are made from scratch and combine culinary creativity with high-quality ingredients.; Fish – another staple of the Finnish diet: Whether it’s tasty salmon soup, fillets of perch, pickled Baltic herring or smoked vendace, the list simply goes on and on. Best consumed fresh at food markets, delis and restaurants throughout the country, fish dishes are the heart of the Finnish diet. Unsure what to try? Go for smoked – it’s the archipelago’s signature way to enjoy the fruits of the sea. Or make some memories on a fishing trip, where you can catch and prepare your own. Contact the local tourist information points for tours. Tip: Crayfish parties, or “rapujuhlat” (“kräftskiva” in Swedish), are a Swedish tradition the Finns adopt during late summer’s crayfish season. These small fresh-water lobsters are considered a gourmet treat, which is why they are feted in style – often accompanied by plenty of schnapps and special crayfish songs. If you’re invited to one of these parties, do say yes!; Fresh, juicy berries are the gold of Finland’s forests: Plentiful and sweet when in season, lingonberries, woodland strawberries and blueberries taste best when they’re picked straight from the forest or purchased fresh by the litre at a local market. Off-season and outside of the warmer months of the year, local jams with cloudberries and the notorious sea-buckthorn are the preferred way to enjoy Finland’s berry bounty. Tip: Enjoy your berries dried and ground. METTÄ Nordic offers the exciting flavours of the Finnish forests in powder form. Buy a bag or two and try this delicious addition on breakfasts and snacks at home!; Mushrooms – ceps, chanterelles, false morel and list goes on..: Under the Everyman’s right, you can pick almost anything your heart desires while visiting Finland’s forests. Just be mindful and check the local rules if roaming in national parks or other protected areas. Tip: Please don’t pick mushrooms unless you’re an experienced forager. Guided mushroom-picking tours are always the safest bet for beginners.; Finnish superfood innovations have exploded onto the world’s markets: From Beanit’s plant-based protein products made from Nordic Fava beans to locally brewed kombucha drinks by Good Guys Kombucha in Pirkkala, Finland’s food companies are always thinking ahead. Tip: Want to try Finland’s superfoods? Go wild with an herb foraging class. You’ll not only return with a bag full of fresh greens, you’ll also get to try forest bathing, another Finnish. Foraging trips can be organised through companies such as Feel the Nature, and Finland, Naturally Experiences.   Hungry for more? Here’s a list of iconic foods and regional delicacies beloved by locals and visitors alike. Make sure to try them all!; Leipäjuusto – delicious dairy product: Leipäjuusto, known as “Squeaky Cheese,” is a mild, incredibly tasty cheese that’s typically made of cow’s milk. First, the milk is curdled, then it’s baked it in the oven, and finally, it’s cut into thin wedges and served. The exterior of the cheese gets its spotted black and white colouring from the heat of the oven, and it’s Finnish name means “cheese bread” (since it’s baked like bread). The end product is a yummy cheese with a deliciously squeaky consistency. Tip: Leipäjuusto is often served with cloudberry jam as dessert, but the traditional Sámi way of eating it is to dip the wedges in hot, black coffee. Try it to experience the perfect blend of smooth, fatty cheese cut by hot, bitter coffee.; Mild but full of flavour, Finnish salmon soup: Finnish salmon soup is a classic that’s served both at home and in restaurants. The most popular version features creamy white broth studded with salmon, onions and potatoes and garnished with a handful of dill. You’ll find this delicacy on the menu at many restaurants and cafés – it’s a true comfort food that will warm you up on a winter’s day.; Endless varieties of new potatoes: New potatoes with herring (silli). New potatoes with fresh lake fish and chantarelle sauce. New potatoes with fish roe (mäti). New potatoes with just a knob of butter, some dill and a little salt. The variations of new potatoes available in Finland are seemingly endless and certainly mouth-watering. You’ll find that Finns can talk about new potatoes forever, as the little spuds hold the promise of summer. Look for them starting around midsummer and expect to see statistics on the harvest in all the local papers.; Poronkäristys from Lapland: Reindeer are found in the northern province of Lapland, and according to recent research, their meat is one of the healthiest you can put on your plate. It’s high in B-12, omega-3 and omega-6 – and it’s lean and delicious! Served alongside mashed potatoes, sautéed reindeer is a Finnish treat that’s eaten throughout the country, all year round.; Kalakukko – a local delicacy: Describing this local delicacy is difficult, but it’s essentially a combination of salty vendace and fatty pork that’s then wrapped in a rye crust and baked in foil. Best served alone or with heaps of butter, this delicacy is most delicious at its origin: a busy marketplace in Kuopio in Finland’s Lakeland. Loosely translated as “fish rooster,” this iconic pie’s name makes absolutely no sense, but it tastes incredibly good! And fun fact: the name of this unique delicacy has been granted protection under the EU quality scheme, along with Karjalanpiirakka, or Karelian pie.; Karjalanpiirakka – a sublime pastry: And speaking of Karjalanpiirakka, Karelian pie is the crown jewel of Finnish cuisine. This sublime pastry is originally from the eastern province of Karelia, and with its filling of delicious rice, potatoes or carrots enclosed in an incredibly crisp rye crust, this buttery delicacy will win you over in no time. It’s best served with plenty of fresh butter or a serving of egg butter-spread. Tip: You’ll find Karelian pies at any self-respecting Finnish marketplace, café, supermarket or gas station. Fresh, unbaked pies can also be purchased and baked at home!; Ruisleipä – made from sour dough: Ruisleipä – or rye bread – is a staple of the Finnish diet. While there are many varieties, the most popular and widely available is reikäleipä, meaning “bread with a hole.” People used to hang their bread on poles from the rafters, and while it’s dense, flat and very heavy, many Finns will actually have it sent through the post while they’re living abroad. Näkkileipä is the cracker version of rye bread and there are many varieties, including the internationally-sold Finn Crisp cracker. These are eaten at breakfast with butter, cheese and other spreads, with soups at lunch or even as an evening snack.; Rieska – Finnish flatbreads: Rieska is made with dough from a variety of local grains that’s then shaped into a chapati-like bread (or “kovaohranen”). If they’re baked with potato dough, they’re called “lepuska.” The magic of this flatbread lies in its freshness. Prepared on the same day they’re consumed, these savoury delights go great with heaps of butter and scorching black coffee. Tip: Try a different flatbread in each place you visit. Hopefully, you’ll have the chance to taste the local Savo variety, which is made with sour milk.; Korvapuusti – a pastry enjoyed with a cup of coffee: Korvapuusti translates to “slapped ears” in English, but these pastries are essentially cinnamon buns. And while Finland doesn’t hold a patent on cinnamon buns, it probably should. Usually enjoyed with a cup of coffee (Finns consume more coffee and, perhaps, more cinnamon buns than any other European nation), it can be difficult to stop at just one. Or two.; Bilberry pie – adored by all: In July and August, bilberries paint the Finnish forest. You’ll want to pick and freeze them for winter (like the Finns do), but these berries are best enjoyed in the summer months, whether on their own or in homemade pies. Although most Finnish berries can be made into pie filling, bilberry pie, or mustikkapiirakka, served with fresh milk, is one that’s adored by all.; Chocolate from Finland: Chocolate isn’t exactly a Finnish invention, but it’s prepared and sold all over the country. One place to enjoy it is at Brunberg, a famous chocolate shop in Porvoo. In operation since 1871, this family-owned and operated business offers a wide variety of tasty treats. Favourites include Truffles and Kisses, which have become hits among those who love high-quality chocolate. Fazer, another family-operated chocolate brand, can be found in any Finnish café, supermarket or kiosk – their products are even sold as souvenirs in the airport’s duty-free shop. The company produces the country’s most iconic chocolate, Fazerin Sininen, which is loosely translated and commonly known as “the Blue.” This chocolate, along with Fazer’s licorices and other sweets, are frequently shipped abroad, and their company-operated cafés are scattered throughout Finland. They serve Karelian pies and salmon soup, too!
        '''
    ),
    Document(
        page_content='''
        Guide to driving in Finland Here are some tips for driving in Finland – rain, snow or shine: While Finland’s roads are typically in good condition, Nordic weather occasionally throws drivers a curveball. Whether you encounter reindeer on the road in Lapland or ice…well, pretty much anywhere, here’s some practical information for a smoother, safer road trip.; Renting a car: There are car hire companies in all major towns and cities, as well as at airports. It’s worth booking in advance, and you must have a credit card on you when you book. If you have an EU or Swiss drivers’ licence, you can hit the road in Finland straight away.; Avoiding traffic: Traffic jams aren’t common in Finland. Although you might hear some complaints about traffic from Finns, it’s rare to hear them from those who live in big cities elsewhere. To a Finn, a few-minute delay is the equivalent of heavy traffic.; Speed limits: Generally, the speed limit in Finland is 50 km/h in built-up areas and 80 km/h outside of them. Both limits are typically enforced if there is no other speed limit posted. On major highways in summer, you can drive 100 km/h. On motorways, the summertime limit is 120 km/h. In winter, however, the speed limit on highways is reduced to 80 km/h, and on motorways, it’s 100 km/h.; Turn your lights on!: If an oncoming vehicle flashes its high beams at you, it usually means you don’t have your headlights on. It’s a law in Finland that drivers must have their lights on – no matter the season and even in the midnight sun. But being flashed could also mean there’s an accident ahead or an animal is on the roadway, so be sure to stay alert.; Paying (or not paying) tolls: There are zero toll roads or bridges in Finland. Yes, zero. That’s because constructing and maintaining roads and highways is mostly funded by Finnish taxes. So go on, enjoy the drive – it’s on us!; Traffic enforcement cameras: Finnish police use automatic traffic surveillance equipment on heavily trafficked roads and in urban areas. You’ll know you’re driving on one of these roads if you see a yellow road sign depicting a camera.; Beware of the animals: Drivers in Finland should stay alert for signs warning of elk crossing the roadways. If you see one, slow down, and at night, use high beams whenever possible. While many of the country’s major highways are lined with high elk fences, accidents do happen – and they can be fatal for drivers, passengers and animals. Also, if you’re driving in Lapland, a reindeer (or twelve!) on the road is a very common sight. Please make sure you drive carefully and adjust your speed to account for limited visibility and potentially dangerous driving conditions.; Essentials of driving in winter: The downside of the delightful winter weather in Finland is that, at times, roads can be very icy. Because of this, all vehicles must be equipped with winter tyres between December 1st and March 1st. You might even want to have studded tires if you’re driving in the northern part of the country. Also, be sure to set aside adequate time for driving in winter. When the roads are icy or there’s a heavy snowfall, the only way to go about it safely is to drive slowly. Note that roads are generally not salted. Instead, they’re cleared by snow ploughs. When driving an electric car, be aware that cold weather and cabin heating cut your driving power. And whether your car is powered by electricity or gasoline, it’s always a good idea to wear or bring warm clothes with you when driving in Finland – just in case you need to make an unexpected stop.
        '''
    ),
]

In [ ]:
# This text splitter is used to create the parent documents
# Set the chunk size to define how large our 'Parent' document is
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

# This text splitter is used to create the child documents that are indexed for similarity search
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# Initialise the FAISS retriever -> Ranked retrieval, Similarity search using Locality Sensitive Hashing with Random Projections and Hamming Distance
# https://python.langchain.com/docs/integrations/vectorstores/faiss/
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world"))) # Initialise FAISS index with the dimensionality

# vectorstore to index the child chunks
vectorstore = FAISS(
    embedding_function=embeddings_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
# storage layer for the parent documents
store = InMemoryStore()
# retriever (empty at the start)
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={"k": 1} # number of docs to return
)

# on retrieval we retrieve the parent chunks
retriever.add_documents(docs, ids=None)

<u>**Test Query 1 for Parent-Document Approach**</u>
- Might need to put all the queries in a list and run a for loop to run a function to output the results for comparison

In [ ]:
test_query_1 = "best accomodation in finland"

In [ ]:
sub_docs = vectorstore.similarity_search(test_query_1)

In [ ]:
sub_docs

[Document(metadata={'doc_id': 'f23e621c-bcbf-48f8-86ba-aedb91b219b0'}, page_content="10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've"),
 Document(metadata={'doc_id': '9a864bbe-f226-4823-94b0-2b8c5bfefe85'}, page_content='visit the historical landmark Olavinlinna Castle and the iconic Punkaharju Ridge.; 5. PihlasResort – Joroinen, Lakeland: Located on the shore of the biggest lake in Finland, Lake Saimaa, PihlasResort is the perfect place to embrace luxury. The brand new suites, opening in summer 2023, are equipped with a kitchen, a private sauna, and a terrace facing a private garden. Relax by the lake, have a'),
 Document(metadata={'doc_id

As we can see above, although relevant chunks are returned, not all information related to the best accomodation in finland is returned, our user could be missing out the accomodation of their choice.

In [ ]:
retrieved_docs = retriever.invoke("best accomodation in finland")

In [ ]:
retrieved_docs

[Document(metadata={}, page_content="10 new unique places to stay in Finland From modern wooden cabins and brand-new glass huts to resorts with a touch of luxury: Finland offers a plethora of unique accommodation options that showcase the country's breathtaking natural beauty both in the materials and the locations. In recent years, there has been upsurge in unique new hotels, resorts, and cabins opening up around the country. We've curated a list of the coolest and most recently opened ones where you can immerse yourself in Finnish nature and find your inner happiness. Article published in June 2023.; 1. Hilltop Forest – Inkoo, Helsinki region: Hilltop Forest, a small resort in Inkoo, offers charming triangular wooden huts with large windows facing a lush forest. Curl up in the fresh linen bedding and admire the view over the majestic pine trees and birches. The sole mission of the day is to stroll through the forest to the nearby spa. The resort's restaurant serves breakfast, lunch, 

We can see above that we now return 'Parent' Documents with more information

In [ ]:
# LLM
llm = HuggingFacePipeline(
      pipeline=pipeline(
        model="Qwen/Qwen2.5-3B-Instruct",
        task="text-generation",
        temperature=0.2,
        do_sample=True,
        repetition_penalty=1.1,
        max_new_tokens=400,
        device_map="auto"
      )
    )

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
# Initialise the template for us to feed the LLM context
template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Question: {question}
Context: {context}
Helpful Answer:
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
output_parser = StrOutputParser()

def print_context_length(inputs):
    context_length = len(inputs['context'][0].page_content)
    print(f"Context length: {context_length}")
    return inputs

chain = (
    {"context": (lambda x: x["question"]) | retriever,
     "question": (lambda x: x["question"])}
    | RunnableLambda(print_context_length)
    | prompt
    | llm.bind(skip_prompt=True)
    | StrOutputParser()
)

**TODO: Use the full data and a number of queries to test if Parent-Document Retrieval improves performance**
- How to compare?

In [ ]:
answer = chain_chunk_parent.invoke({"question":"Please provide me with the best accomodation in Finland"})

Context length: 1983
The inference time using the context as the chunk parent document is 174.22468638420105


In [ ]:
answer

'Based on the information provided in the context, it seems like the best accommodations in Finland could vary depending on personal preferences such as location, style, and amenities. However, two notable examples mentioned are:\n\n1. **Hilltop Forest - Inkoo**: This is described as a charming resort offering triangular wooden huts with large windows, set amidst a lush forest near Helsinki. It includes a nearby spa and features a variety of dining options.\n\n2. **The Torby - Fiskars village**: A newly opened boutique hotel located in the picturesque Fiskars village, known for its creative atmosphere, including art studios, shops, and gardens. It provides a tranquil setting for travelers looking for inspiration and cultural experiences.\n\nFor the best recommendation, I would need more specific details about what type of accommodation suits your needs (e.g., budget, preferred activities, proximity to attractions) and interests. Based solely on the given information, these two seem par